In [1]:
%pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 10.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 10.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 5.0 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 5.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 5.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 572.6/572.6 kB 4.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.7/33.7 MB 5.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 890.1/890.1 MB 2.7 MB/s eta 0:00:0000:0100:03
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.3/24.3 MB 9.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.5/123.5 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━

In [10]:
!tar -xf car_color_dataset.zip

In [98]:
from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns

# Augmentation


In [10]:
def color2label(color):
    c2l = {
        'Black': 0,
        'Blue': 1,
        'Brown': 2,
        'Cyan': 3,
        'Green': 4,
        'Grey': 5,
        'Orange': 6,
        'Red': 7,
        'Violet': 8,
        'White': 9,
        'Yellow': 10
    }

    return c2l[color]
def label2color(label):
    c2l = {
        'Black': 0,
        'Blue': 1,
        'Brown': 2,
        'Cyan': 3,
        'Green': 4,
        'Grey': 5,
        'Orange': 6,
        'Red': 7,
        'Violet': 8,
        'White': 9,
        'Yellow': 10
    }
    l2c = {v: k for k, v in c2l.items()}
    return l2c[label]

# Class balance

In [11]:
import numpy as np
import pandas as pd

import albumentations as A
import cv2
from albumentations.pytorch import ToTensorV2

import torch
import torchvision

import os
from tqdm.auto import tqdm
import gc

In [12]:
def label_weight(label):
    lw = {
        0: 2,
        1: 3,
        2: 8,
        3: 3,
        4: 3,
        5: 2,
        6: 1,
        7: 1,
        8: 8,
        9: 2,
        10: 2
    }

    return lw[label]

In [13]:
class CarsDataset2(torch.utils.data.Dataset):
    def __init__(self, path_to_data, size, transform=False, train=False):
        self.path_to_data = path_to_data
        self.transform = transform
        self.train = train
        self.data = []
        self.labels =[]
        self.size =size
        self._get_data_from_path()
        if train:
            
            self._add_train_augmentations()
            #self._add_augmentations()
            self._final_augs()

    def _get_data_from_path(self):
        for img in self.path_to_data:
            label = color2label(os.path.normpath(img).split(os.path.sep)[1])
            img = cv2.imread(img)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            self.data.append(img)
            self.labels.append(label)
            del img
                
                

    def _add_train_augmentations(self):
        data = []
        labels=[]
        for i in tqdm(range(len(self.data))):
            img = self.data[i]
            label = self.labels[i]
            weight = label_weight(label)
            for i in range(weight):
                transformed = self.transform(image=img)
                img_tensor = transformed['image']
                data.append(img_tensor)
                del img_tensor
                del transformed
                labels.append(label)
            del img
        self.data += data
        self.labels+=labels
        self.df = pd.DataFrame(self.labels)
        self.class_counts = list(self.df[0].value_counts().sort_index())
        del data
        gc.collect()
    
    def _add_augmentations(self):
        size=self.size
        self.transform = A.Compose([
            A.OneOf([
                A.Blur(blur_limit=2.5, p=0.75),
                A.MotionBlur(blur_limit=(3, 5), p=0.75),
            ], p=0.75),
            A.OneOf([
                A.Affine(shear=21, mode=cv2.BORDER_REPLICATE, p=0.75),
                A.Compose([
                    A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05,
                                       rotate_limit=16, border_mode=cv2.BORDER_REPLICATE, p=0.75),
                    A.GridDistortion(distort_limit=0.1, border_mode=cv2.BORDER_REPLICATE, p=0.75),
                ], p=0.75)
            ], p=0.75),
            A.OneOf([
                A.Cutout(num_holes=12, max_h_size=21, max_w_size=21, p=0.75),
                A.Cutout(num_holes=6, max_h_size=42, max_w_size=42, p=0.75),
            ], p=0.75),
            A.CLAHE(clip_limit=2.5, p=0.75),
            A.ImageCompression(quality_lower=90, quality_upper=100, p=0.75),
        ])
        data = []
        labels=[]
        for i in tqdm(range(len(self.data))):
            img = self.data[i]
            label = self.labels[i]
            transformed = self.transform(image=img)
            img_tensor = transformed['image']
            data.append(img_tensor)
            del img_tensor
            del transformed
            labels.append(label)
            del img
        self.data+=data
        self.labels+=labels
        self.df = pd.DataFrame(self.labels)
        self.class_counts = list(self.df[0].value_counts().sort_index())
        del data
    def _final_augs(self):
        size=self.size
        self.transform = A.Compose([
            A.LongestMaxSize(size),
            A.PadIfNeeded(size, size),
            A.Normalize(p=1.0),
            ToTensorV2()
        ])
        data = []
        labels=[]
        for i in tqdm(range(len(self.data))):
            img = self.data[i]
            label = self.labels[i]
            transformed = self.transform(image=img)
            img_tensor = transformed['image']
            data.append(img_tensor)
            del img_tensor
            del transformed
            labels.append(label)
            del img
        del self.data
        self.data=data
        self.labels=labels
        del data
    def __getitem__(self, idx):
        return (self.data[idx], self.labels[idx])
        
    def __len__(self):
        return len(self.data)

In [14]:
class ValDataset(torch.utils.data.Dataset):
    def __init__(self, path_to_data, size, transform=False):
        self.path_to_data = path_to_data
        self.transform = transform
        self.data = []
        self.labels =[]
        self.size =size
        self._get_data_from_path()
        self._final_augs()

    def _get_data_from_path(self):
        for img in self.path_to_data:
            label = color2label(os.path.normpath(img).split(os.path.sep)[1])
            img = cv2.imread(img)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            self.data.append(img)
            self.labels.append(label)
            del img
                
    def _final_augs(self):
        size=self.size
        self.transform = A.Compose([
            A.LongestMaxSize(size),
            A.PadIfNeeded(size, size),
            A.Normalize(p=1.0),
            ToTensorV2()
        ])
        data = []
        labels=[]
        for i in tqdm(range(len(self.data))):
            img = self.data[i]
            label = self.labels[i]
            transformed = self.transform(image=img)
            img_tensor = transformed['image']
            data.append(img_tensor)
            del img_tensor
            del transformed
            labels.append(label)
            del img
        del self.data
        self.data=data
        self.labels=labels
        del data
        
    def __getitem__(self, idx):
        return (self.data[idx], self.labels[idx])
        
    def __len__(self):
        return len(self.data)

In [15]:
from torch.utils.data import DataLoader

In [16]:
path_to_train = './train'
size = 256

In [17]:
imgs=np.array([])
labels=np.array([],dtype=int)
for folder in tqdm(os.listdir(path_to_train)):
            img_folder = os.path.join(path_to_train, folder)
            for img in os.listdir(img_folder):
                imgs = np.append(imgs, os.path.join(path_to_train, folder,img))
                labels=np.append(labels,color2label(folder))

  0%|          | 0/11 [00:00<?, ?it/s]

In [18]:
train_transform = A.Compose([
    A.LongestMaxSize(size),
    A.PadIfNeeded(size, size),
    A.HorizontalFlip(p=0.5),
])

In [19]:
labels

array([2, 2, 2, ..., 4, 4, 4])

In [20]:
from sklearn.model_selection import StratifiedShuffleSplit
imgs_train = np.array([],dtype=int)
imgs_val = np.array([],dtype=int)
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42) 
for train_index, test_index in sss.split(imgs, labels):
    imgs_train = np.append(imgs_train, imgs[train_index])
    imgs_val = np.append(imgs_val, imgs[test_index])

In [21]:
imgs_train.shape, imgs_val.shape

((3041,), (761,))

In [22]:
train_dataset = CarsDataset2(imgs_train, size,transform=train_transform, train=True)
train_dataloader = DataLoader(train_dataset, batch_size=16, drop_last=True, shuffle=True)
gc.collect()

  0%|          | 0/3041 [00:00<?, ?it/s]

  0%|          | 0/10413 [00:00<?, ?it/s]

19

In [23]:
val_dataset = ValDataset(imgs_val, size)
val_dataloader = DataLoader(val_dataset, batch_size=16, drop_last=True, shuffle=False)
gc.collect()

  0%|          | 0/761 [00:00<?, ?it/s]

19

In [24]:
train_dataset.class_counts

[966, 908, 1080, 856, 860, 798, 890, 842, 1170, 987, 1056]

# Adding MixPrecision, MADGRAD, lr_scheduler

In [25]:
import math
import torch
from typing import TYPE_CHECKING, Any, Callable, Optional

In [26]:
if TYPE_CHECKING:
    from torch.optim.optimizer import _params_t
else:
    _params_t = Any

In [27]:
from sklearn.metrics import f1_score

In [28]:
def train_one_epoch(model, train_dataloader, criterion, optimizer, scaler, device="cuda:0"):
    model.train()
    total_loss = 0
    total_accuracy = 0
    n_batchs = 0
    for X_batch, y_batch in train_dataloader:
        optimizer.zero_grad()
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        with torch.autocast(device_type='cuda', dtype=torch.float16):
            y_preds = model(X_batch)
            loss = criterion(y_preds, y_batch)
            total_loss += loss
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        total_accuracy += f1_score(y_batch.cpu().numpy(), torch.argmax(y_preds, dim=-1).detach().cpu().numpy(),average='macro')
        n_batchs += 1
    total_accuracy /= n_batchs
    total_loss /= n_batchs
    return total_accuracy, total_loss

def predict(model, val_dataloader, criterion, device="cuda:0"):
    model.eval()
    predicted_classes = torch.tensor([])
    true_classes = torch.tensor([])
    total_loss = 0
    n_batchs = 0
    with torch.no_grad():
        for X_batch, y_batch in val_dataloader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            y_preds = model(X_batch)
            total_loss += criterion(y_preds, y_batch)
            y_batch = y_batch.cpu()
            predicts = model(X_batch)
            predicts = torch.argmax(predicts, dim=1).cpu()
            predicted_classes = torch.cat((predicted_classes, predicts), dim=-1)
            true_classes = torch.cat((true_classes, y_batch), dim=-1)
            n_batchs += 1
    total_loss /= n_batchs
    return total_loss, predicted_classes.numpy(), true_classes.numpy()

def train(model, train_dataloader, val_dataloader, criterion, optimizer, scaler, lr_scheduler, device="cuda:0", n_epochs=10):
    model.to(device)
    for epoch in tqdm(range(n_epochs)):
        train_acc, train_loss = train_one_epoch(model, train_dataloader, criterion, optimizer, scaler, device=device)
        if lr_scheduler is not None: 
            lr_scheduler.step()
        val_loss, predicted_classes, true_classes = predict(model, val_dataloader, criterion, device=device)
        val_acc = f1_score(true_classes, predicted_classes,average='macro')
        print(f'Train Loss:{train_loss} Train Accuracy:{train_acc}')
        print(f'Val Loss:{val_loss} Val Accuracy:{val_acc} ')
        del true_classes, predicted_classes

In [29]:
from torch import nn

from torch.cuda.amp import GradScaler

In [30]:
!pip install madgrad
from madgrad import MADGRAD

In [31]:
class CarsDatasetTest(torch.utils.data.Dataset):
    def __init__(self, path_to_data, transform=False, train=False):
        self.path_to_data = path_to_data
        self.transform = transform
        self.train = train
        self.data = []
        self._get_data_from_path()
        if self.transform:
            self._add_augmentations()
        #self.data = np.array(self.data)

    def _get_data_from_path(self):
            for img in tqdm(os.listdir(self.path_to_data)):
                if img[0]!='.':
                    img = cv2.imread(os.path.join(self.path_to_data, img))
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                    label = 0
                    self.data.append((img, label))

    def _add_augmentations(self):
        data = self.data.copy()
        del self.data
        self.data = []
        for img, label in tqdm(data):
            transformed = self.transform(image=img)
            img_tensor = transformed['image']
            self.data.append((img_tensor, 0))
        del data

    def __getitem__(self, idx):
        return self.data[idx]
        
    def __len__(self):
        return len(self.data)

In [32]:
val_transform = A.Compose([ 
    A.LongestMaxSize(size),
    A.PadIfNeeded(size, size),
    A.Normalize(), 
    ToTensorV2()])

In [33]:
path_to_test = './public_test'
test_dataset = CarsDatasetTest(path_to_test, transform=val_transform)
test_dataloader = DataLoader(test_dataset, batch_size=16)
#val_dataloader = DataLoader(val_dataset, batch_size=16)

  0%|          | 0/1436 [00:00<?, ?it/s]

  0%|          | 0/1436 [00:00<?, ?it/s]

In [34]:
def predict(model, val_dataloader, criterion, device="cuda:0"):
    model.eval()
    predicted_classes = torch.tensor([])
    true_classes = torch.tensor([])
    total_loss = 0
    n_batchs = 0
    with torch.no_grad():
      for X_batch, y_batch in tqdm(val_dataloader):
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        X_batch_hf = torch.flip(X_batch, dims=(2,))
        X_batch_vf = torch.flip(X_batch, dims=(1,))
        y_preds = model(X_batch)
        y_preds += model(X_batch_hf)
        #y_preds += model(X_batch_vf) 
        y_preds /= 2
        total_loss += criterion(y_preds, y_batch)
        y_batch = y_batch.cpu()
        predicts = model(X_batch)
        predicts = torch.argmax(predicts, dim=1).cpu()
        predicted_classes = torch.cat((predicted_classes, predicts), dim=0)
        true_classes = torch.cat((true_classes, y_batch), dim=0)
        n_batchs += 1
      total_loss /= n_batchs
      return total_loss, predicted_classes.numpy(), true_classes.numpy()

In [35]:
%pip -q install optuna

In [41]:
import optuna

from torchvision.models import resnext101_32x8d

device="cuda:0"

In [42]:
def penis(trial):
    model = resnext101_32x8d(weights="IMAGENET1K_V2")
    model.fc = nn.Linear(2048, 11)
    model.to(device)
    criterion = nn.CrossEntropyLoss()
    madgrad_params={
        "params":model.parameters(),
        "lr":trial.suggest_float("lr", 1e-8, 1e-5, log=False),
        "weight_decay":trial.suggest_float("weight_decay", 1e-6, 1e-3, log=False),
        "momentum":trial.suggest_float("momentum", 0.7, 0.99, log=False)
    }
    scaler = GradScaler()
    n_epochs = trial.suggest_int("n_epochs", 3, 10)
    optimizer = MADGRAD(**madgrad_params)
    scheduler_params={
        "optimizer":optimizer,
        "T_max": n_epochs,
        "eta_min": trial.suggest_float("eta_min", 1e-10, 1e-6, log=False),
        "last_epoch":-1, 
        "verbose":False
        
    }
    lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(**scheduler_params)
    train(model, train_dataloader, val_dataloader, criterion, optimizer, scaler, lr_scheduler, device="cuda:0",n_epochs=n_epochs)
    total_loss, predicted_classes, true_classes = predict(model, val_dataloader, criterion)
    return f1_score(true_classes,predicted_classes , average="macro")

In [43]:
torch.cuda.empty_cache()

In [44]:
study = optuna.create_study(direction="maximize")
study.optimize(penis, n_trials=10,show_progress_bar=False)

print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2022-11-23 11:47:33,736] A new study created in memory with name: no-name-ec439c93-7e19-4e45-b755-d4cc74b36562


  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/47 [00:00<?, ?it/s]

Train Loss:0.5509999990463257 Train Accuracy:0.8320083444940252
Val Loss:0.10124314576387405 Val Accuracy:0.9669457656984369 


  0%|          | 0/47 [00:00<?, ?it/s]

...

Train Loss:0.04684940353035927 Train Accuracy:0.9809555203242009
Val Loss:0.10023083537817001 Val Accuracy:0.9714811046403002 


  0%|          | 0/47 [00:00<?, ?it/s]

Train Loss:0.021374978125095367 Train Accuracy:0.990192433316243
Val Loss:0.07262174785137177 Val Accuracy:0.9742271419131463 


[W 2022-11-23 13:19:48,267] Trial 4 failed because of the following error: KeyboardInterrupt()
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/optuna/study/_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_16695/301993058.py", line 24, in penis
    train(model, train_dataloader, val_dataloader, criterion, optimizer, scaler, lr_scheduler, device="cuda:0",n_epochs=n_epochs)
  File "/tmp/ipykernel_16695/53860888.py", line 45, in train
    train_acc, train_loss = train_one_epoch(model, train_dataloader, criterion, optimizer, scaler, device=device)
  File "/tmp/ipykernel_16695/53860888.py", line 13, in train_one_epoch
    scaler.scale(loss).backward()
  File "/opt/conda/lib/python3.10/site-packages/torch/_tensor.py", line 487, in backward
    torch.autograd.backward(
  File "/opt/conda/lib/python3.10/site-packages/torch/autograd/__init__.py", line 197, in backward
    Variable._execution_engine.run_backward(  #

KeyboardInterrupt: 

# Predict

In [500]:
total_loss, predicted_classes, true_classes = predict(model, test_dataloader, criterion)
predicted_classes = np.array([label2color(int(preds)) for preds in predicted_classes])
imgs=[]
for img in tqdm(os.listdir(path_to_test)):
    if img[0]!='.':
        imgs.append(int(os.path.splitext(img)[0]))
test_df = pd.DataFrame(predicted_classes, index=imgs)
test_df = test_df.sort_index()
test_df = test_df.rename(columns={0: test_df[0][0]})
test_df.drop([0]).reset_index().drop(columns=['index']).to_csv('subm.csv',index=False)

  0%|          | 0/90 [00:00<?, ?it/s]

  0%|          | 0/1436 [00:00<?, ?it/s]